In [2]:
# ==============================
# 02_数据清洗（优化版）
# ==============================
import pandas as pd
import sqlite3

# === 0. 读取数据 ===
file_path = r'E:\shenzhen_house_price_project\data\clean\合并数据.xlsx'
df = pd.read_excel(file_path)
print("原始数据规模：", df.shape)
print("="*100)

# === 1. 去重 ===
print("重复行数量：", df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
print("去重后数据规模：", df.shape)
print("="*100)

# === 2. 删除无用字段 ===
drop_cols = ['title', 'address', 'label']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# === 3. 修改列名（去除单位、空格替换下划线） ===
df.columns = df.columns.str.replace(r'（.*?）|\(.*?\)', '', regex=True)
df.columns = [c.strip().replace(' ', '_') for c in df.columns]
print("列名清洗完成：", df.columns.tolist())
print("="*100)

# === 4. 查看异常值情况 ===
print("卧室数分布：")
print(df['roomnum'].value_counts(dropna=False))
print('-'*60)

print("楼层缺失数量：", df['C_floor'].isna().sum())
print('-'*60)

# === 5. 删除异常值 ===
# 卧室数不能为 0
df = df[df['roomnum'] != 0]

# 楼层不能为空或空白
df = df[df['C_floor'].notna()]
df = df[df['C_floor'].astype(str).str.strip() != '']

df = df.reset_index(drop=True)
print("清洗完成后数据量：", len(df))
print("="*100)

# === 6. 拼音 -> 中文转换 ===
dist_dict = {
    'longgang': '龙岗', 'longhua': '龙华', 'baoan': '宝安',
    'nanshan': '南山', 'futian': '福田', 'luohu': '罗湖',
    'pingshan': '坪山', 'guangming': '光明', 'yantian': '盐田', 'dapengxinqu': '大鹏新区'
}

df['district'] = df['district'].apply(lambda x: dist_dict.get(x, x))

# === 7. 单位换算 ===
# 单价：元/平 -> 万元/平
df['per_price'] = df['per_price'] / 10000

# === 8. 查看关键字段描述 ===
print(df[['roomnum', 'hall', 'AREA', 'per_price', 'total_price']].describe())
print("="*100)

# === 9. 查看分类变量取值 ===
con = sqlite3.connect(':memory:')
df.to_sql('df', con, index=False)

for col in ['district', 'roomnum', 'hall', 'C_floor', 'school', 'subway']:
    print(f"字段：{col}")
    print(pd.read_sql_query(f"SELECT DISTINCT {col} FROM df", con))
    print('-'*20)
print("="*100)

# === 10. 生成列名含义表 ===
save_path = r'E:\shenzhen_house_price_project\data\clean'
table_columns = pd.DataFrame(columns=['表名','表说明','列名','列名含义','备注'])
table_columns['列名'] = df.columns.tolist()
table_columns.to_excel(save_path + r'\列名含义分析表（请另存）.xlsx', index=False)
print("列名含义分析表 已生成")
print("="*100)

# === 11. 保存清洗后数据 ===
df.to_excel(save_path + r'\清洗后数据.xlsx', index=False)
print("已保存：清洗后数据.xlsx")


原始数据规模： (46945, 13)
重复行数量： 15064
去重后数据规模： (31881, 13)
列名清洗完成： ['district', 'roomnum', 'hall', 'AREA', 'C_floor', 'school', 'subway', 'per_price', 'total_price', 'source_file']
卧室数分布：
roomnum
3.0    13809
4.0     6548
2.0     5819
1.0     2175
5.0     1932
NaN      910
6.0      426
7.0      145
8.0       77
9.0       36
0.0        4
Name: count, dtype: int64
------------------------------------------------------------
楼层缺失数量： 1973
------------------------------------------------------------
清洗完成后数据量： 29904
            roomnum         hall          AREA     per_price   total_price
count  29904.000000  29904.00000  29904.000000  29904.000000  29904.000000
mean       3.073067      1.80307    103.081402      5.695858    682.099839
std        1.095166      0.47922     52.367763      3.555464    870.308192
min        1.000000      0.00000     13.870000      0.620300     37.800000
25%        2.000000      2.00000     76.000000      3.429800    268.000000
50%        3.000000      2.00000     89